In [ ]:
import math
import os
import random
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.fft import fft, ifft, fftfreq
from scipy.signal import firwin, freqz, lfilter, welch
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

!pip install mne
!pip install moabb
!pip install braindecode

import mne
import moabb
from mne.decoding import CSP
from moabb.datasets import BNCI2014_001
from moabb.evaluations import WithinSessionEvaluation
from moabb.paradigms import LeftRightImagery

In [ ]:
from braindecode.datasets import MOABBDataset

In [ ]:
dataset = MOABBDataset(dataset_name="BNCI2014_001", subject_ids = [i for i in range(1, 10)])

In [ ]:
from braindecode.preprocessing import create_windows_from_events

trial_start_offset_seconds = -0.5
# Extract sampling frequency, check that they are same in all datasets
sfreq = dataset.datasets[0].raw.info["sfreq"]
assert all([ds.raw.info["sfreq"] == sfreq for ds in dataset.datasets])
# Calculate the trial start offset in samples.
trial_start_offset_samples = int(trial_start_offset_seconds * sfreq)

# Create windows using braindecode function for this. It needs parameters to define how
# trials should be used.
windows_dataset = create_windows_from_events(
    dataset,
    trial_start_offset_samples=trial_start_offset_samples,
    trial_stop_offset_samples=0,
    preload=True,
)

In [ ]:
splitted = windows_dataset.split("session")
train_set = splitted['0train']  # Session train
test_set = splitted['1test']  # Session evaluation

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

In [ ]:
batch_size = 128
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size)

In [ ]:
progress_bar = tqdm(enumerate(train_loader), total=len(train_loader))
for batch_idx, (X, y, _) in progress_bar:
  print(X.shape, y.shape)
  print(y)
  break

In [ ]:
class EEGNetLSTM(nn.Module):
    def __init__(self, n_classes=4, in_channels=26,
                 F1=8, D=2, F2=16, kernel_length=64,
                 dropout=0.25, lstm_units=64):
        super(EEGNetLSTM, self).__init__()

        # Block 1: Spatial-temporal features
        self.conv1 = nn.Conv2d(1, F1, (1, kernel_length),
                              padding=(0, kernel_length//2),
                              bias=False)
        self.bn1 = nn.BatchNorm2d(F1)
        self.depthwise = nn.Conv2d(F1, F1*D, (in_channels, 1),
                                  groups=F1, bias=False)
        self.bn2 = nn.BatchNorm2d(F1*D)
        self.elu1 = nn.ELU()
        self.pool1 = nn.AvgPool2d((1, 4))
        self.drop1 = nn.Dropout(dropout)

        # Block 2: Temporal compression
        self.sep_conv = nn.Conv2d(F1*D, F1*D, (1, 16),
                                 padding=(0, 8),
                                 groups=F1*D, bias=False)
        self.pointwise = nn.Conv2d(F1*D, F2, (1, 1), bias=False)
        self.bn3 = nn.BatchNorm2d(F2)
        self.elu2 = nn.ELU()
        self.pool2 = nn.AvgPool2d((1, 8))
        self.drop2 = nn.Dropout(dropout)

        # LSTM
        self.lstm = nn.LSTM(
            input_size=F2,
            hidden_size=lstm_units,
            num_layers=1,
            batch_first=True
        )
        self.classifier = nn.Linear(lstm_units, n_classes)

    def forward(self, x):
        x = x.unsqueeze(1)

        # Block 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.depthwise(x)
        x = self.bn2(x)
        x = self.elu1(x)
        x = self.pool1(x)
        x = self.drop1(x)

        # Block 2
        x = self.sep_conv(x)
        x = self.pointwise(x)
        x = self.bn3(x)
        x = self.elu2(x)
        x = self.pool2(x)
        x = self.drop2(x)

        x = x.squeeze(2)
        x = x.permute(0, 2, 1)

        # LSTM
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        return self.classifier(x)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EEGNetLSTM().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=5, verbose=True
)

def train(model, train_loader, test_loader, epochs=100):
    for epoch in range(epochs):
        model.train()
        train_loss, correct, total = 0, 0, 0
        for X, y, _ in train_loader:
            X, y = X.to(device), y.to(device)

            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += y.size(0)
            correct += predicted.eq(y).sum().item()

        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for X, y, _ in test_loader:
                X, y = X.to(device), y.to(device)
                outputs = model(X)
                loss = criterion(outputs, y)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += y.size(0)
                val_correct += predicted.eq(y).sum().item()

        train_acc = 100 * correct / total
        val_acc = 100 * val_correct / val_total
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(test_loader)

        scheduler.step(val_acc)

        print(f"Epoch {epoch+1}/{epochs} | "
              f"Train: Loss {avg_train_loss:.4f} Acc {train_acc:.2f}% | "
              f"Val: Loss {avg_val_loss:.4f} Acc {val_acc:.2f}% | "
              f"LR: {optimizer.param_groups[0]['lr']:.6f}")

train(model, train_loader, test_loader, epochs=100)

In [ ]:
import torch
import torch.nn as nn

class EnhancedEEGNetLSTM(nn.Module):
    def __init__(self, n_classes=4, in_channels=26,
                 F1=16, D=2, F2=32, kernel_length=64,
                 dropout=0.3, lstm_units=128, bidirectional=True):
        super(EnhancedEEGNetLSTM, self).__init__()

        # Block 1
        self.conv1 = nn.Conv2d(1, F1, (1, kernel_length),
                               padding=(0, kernel_length//2),
                               bias=False)
        self.bn1 = nn.BatchNorm2d(F1)
        self.depthwise = nn.Conv2d(F1, F1*D, (in_channels, 1),
                                   groups=F1, bias=False)
        self.bn2 = nn.BatchNorm2d(F1*D)
        self.elu1 = nn.ELU()
        self.pool1 = nn.AvgPool2d((1, 4))
        self.drop1 = nn.Dropout(dropout)

        # Block 2
        self.sep_conv = nn.Conv2d(F1*D, F1*D, (1, 16),
                                  padding=(0, 8),
                                  groups=F1*D, bias=False)
        self.pointwise = nn.Conv2d(F1*D, F2, (1, 1), bias=False)
        self.bn3 = nn.BatchNorm2d(F2)
        self.elu2 = nn.ELU()
        self.pool2 = nn.AvgPool2d((1, 8))
        self.drop2 = nn.Dropout(dropout)

        # LSTM
        self.lstm = nn.LSTM(
            input_size=F2,
            hidden_size=lstm_units,
            num_layers=2,
            bidirectional=bidirectional,
            batch_first=True,
            dropout=0.2 if bidirectional else 0
        )

        # Dynamic classifier input features
        lstm_output_size = lstm_units * 2 if bidirectional else lstm_units
        self.attention = nn.Sequential(
            nn.Linear(lstm_output_size, lstm_output_size),
            nn.Tanh(),
            nn.Linear(lstm_output_size, 1, bias=False)
        )
        self.drop3 = nn.Dropout(dropout)
        self.classifier = nn.Linear(lstm_output_size, n_classes)

    def forward(self, x):
        x = x.unsqueeze(1)

        # Block 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.depthwise(x)
        x = self.bn2(x)
        x = self.elu1(x)
        x = self.pool1(x)
        x = self.drop1(x)

        # Block 2
        x = self.sep_conv(x)
        x = self.pointwise(x)
        x = self.bn3(x)
        x = self.elu2(x)
        x = self.pool2(x)
        x = self.drop2(x)

        # LSTM
        x = x.squeeze(2)
        x = x.permute(0, 2, 1)

        # LSTM processing
        x, _ = self.lstm(x)

        # Attention mechanism
        attn_weights = torch.softmax(self.attention(x).squeeze(2), dim=1)
        x = torch.sum(x * attn_weights.unsqueeze(-1), dim=1)

        # Classification
        x = self.drop3(x)
        return self.classifier(x)

In [ ]:
model = EnhancedEEGNetLSTM().to(device)

optimizer = torch.optim.AdamW(model.parameters(),
                             lr=0.001,
                             weight_decay=0.01)
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',
    factor=0.5,
    patience=8,
    verbose=True
)

In [ ]:
train(model, train_loader, test_loader, epochs=100)

# Results:
max accuracy on validation set is 70.14% and it was achived by EnhancedEEGNetLSTM on Epoch 75